In [1]:

import threading
import time
import queue
import random

class VegetableMarket:
    def __init__(self, capacity):
        self.capacity = capacity
        self.vegetable_queue = queue.Queue(maxsize=capacity)
        self.stop_simulation = False
        self.lock = threading.Lock()

    def farmer_produce(self, farmer_id, vegetable_type):
        while not self.stop_simulation:
            time.sleep(random.uniform(1, 3))
            with self.lock:
                if not self.vegetable_queue.full():
                    self.vegetable_queue.put((farmer_id, vegetable_type))
                    print(f"Farmer {farmer_id} produced {vegetable_type}")

    def consumer_purchase(self, consumer_id, desired_vegetable):
        while not self.stop_simulation:
            with self.lock:
                try:
                    farmer_id, vegetable_type = self.vegetable_queue.get(timeout=1)
                    if vegetable_type == desired_vegetable:
                        print(
                            f"Consumer {consumer_id} purchased {vegetable_type} from Farmer {farmer_id}"
                        )
                        return
                    else:
                        print(
                            f"Consumer {consumer_id} is waiting for {desired_vegetable}"
                        )
                except queue.Empty:
                    pass

market = VegetableMarket(capacity=5)

farmers = []
for farmer_id in range(3):
    vegetable_type = random.choice(["Tomato", "Carrot", "Cabbage"])
    farmer_thread = threading.Thread(
        target=market.farmer_produce, args=(farmer_id, vegetable_type)
    )
    farmers.append(farmer_thread)
    farmer_thread.start()

consumers = []
for consumer_id in range(5):
    desired_vegetable = random.choice(["Tomato", "Carrot", "Cabbage"])
    consumer_thread = threading.Thread(
        target=market.consumer_purchase, args=(consumer_id, desired_vegetable)
    )
    consumers.append(consumer_thread)
    consumer_thread.start()

time.sleep(20)

market.stop_simulation = True

for farmer_thread in farmers:
    farmer_thread.join()

for consumer_thread in consumers:
    consumer_thread.join()

print("Simulation finished.")



Farmer 1 produced Tomato
Farmer 2 produced Tomato
Consumer 3 purchased Tomato from Farmer 1
Consumer 4 purchased Tomato from Farmer 2
Farmer 0 produced Cabbage
Simulation finished.
